# Import de librairies

In [1]:
import pandas as pd
import numpy as np

In [5]:
x_train=pd.read_csv('xtrain.csv')
x_test=pd.read_csv('xtest.csv')
y_train=pd.read_csv('ytrain_NpxebDC.csv')

/Users/juanfelipearias/Library/Python/3.9/lib/python/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (1,3,5,11,20,49,52,54,55,56,64,70,71,94,95,101,102,114,129,131,132,133,138,139,152,157,163,174,180,184,193,196,201,207,208,219,220,228,241,245,250,256,258,264,265,269,270,271,272,273) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [6]:
y_train

,timestamp,S7-T1,S2-T1,S19-T1,S56-T3,S85-T3,S16-T3,S16-T1,S94-T3,S28-T1,...,S47-T2,S65-T2,S32-T2,S21-T2,S13-T2,S97-T2,S25-T1,S25-T2,S98-T2,S99-T2
0,2019-11-25T00:00:00+00:00,Down,Down,Available,Down,Down,Offline,Offline,Offline,Available,...,Down,Offline,Offline,Available,Available,NaN,NaN,NaN,NaN,NaN
1,2019-11-25T00:15:00+00:00,Down,Down,Available,Down,Down,Offline,Offline,Offline,Available,...,Down,Offline,Offline,Available,Available,NaN,NaN,NaN,NaN,NaN
2,2019-11-25T00:30:00+00:00,Down,Down,Available,Down,Down,Offline,Offline,Offline,Available,...,Down,Offline,Offline,Available,Available,NaN,NaN,NaN,NaN,NaN
3,2019-11-25T00:45:00+00:00,Down,Down,Available,Down,Down,Offline,Offline,Offline,Available,...,Down,Offline,Offline,Available,Available,NaN,NaN,NaN,NaN,NaN
4,2019-11-25T01:00:00+00:00,Down,Down,Available,Down,Down,Offline,Offline,Offline,Available,...,Down,Offline,Offline,Available,Available,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31227,2020-11-08T22:45:00+00:00,NaN,Available,NaN,Available,NaN,Available,Available,Available,Available,...,NaN,NaN,Available,Available,Available,Available,Available,Available,Available,Available
31228,2020-11-08T23:00:00+00:00,NaN,Available,NaN,Available,NaN,Available,Available,Available,Available,...,NaN,NaN,Available,Available,Available,Available,Available,Available,Available,Available
31229,2020-11-08T23:15:00+00:00,NaN,Available,NaN,Available,NaN,Available,Available,Available,Available,...,NaN,NaN,Available,Available,Available,Available,Available,Available,Available,Available
31230,2020-11-08T23:30:00+00:00,NaN,Available,NaN,Available,NaN,Available,Available,Available,Available,...,NaN,NaN,Available,Available,Available,Available,Available,Available,Available,Available
